In [1]:
from time import sleep

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.firefox.service import Service
from tqdm.auto import tqdm
from webdriver_manager.firefox import GeckoDriverManager

In [2]:
df = pd.read_excel("banks_list.xlsx").rename(columns={"REGN": "id"})

In [3]:
df.head()

,id,Наименование полное,Наименование,name_banki_ru,Наименование на английском,url_bank,Банковский идентификационный номер,Код налогоплательщика,Код ОКАТО,Код ОКТМО,СПАРК код,url_sravni,url_irec,url_otz,search_name
0,2306,"АКЦИОНЕРНЫЙ КОММЕРЧЕСКИЙ БАНК ""АБСОЛЮТ БАНК"" (...","АБСОЛЮТ БАНК (ПАО), АКБ",Абсолют Банк,"ABSOLYUT BANK (PAO), AKB",https://www.banki.ru/services/responses/bank/a...,44525976.0,7.736047e+09,4.528657e+10,4.537900e+10,1112755.0,https://www.sravni.ru/bank/absoljut-bank/,https://irecommend.ru/content/absolyut-bank,https://otzovik.com/reviews/bank_absolyut_bank...,"Абсолют, Абсолют банк"
1,2879,"АКЦИОНЕРНЫЙ КОММЕРЧЕСКИЙ БАНК ""АВАНГАРД"" - ПУБ...","АВАНГАРД, ПАО АКБ",Авангард,"AVANGARD, PAO AKB",https://www.banki.ru/services/responses/bank/a...,44525201.0,7.702021e+09,4.528660e+10,4.538400e+10,1143956.0,https://www.sravni.ru/bank/avangard/,https://irecommend.ru/content/avangard,https://otzovik.com/reviews/bank_avangard_russia/,"Авангард, Авангард банк"
2,415,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ БАНК ...,"АВЕРС, ООО БАНК",Аверс,"AVERS, OOO BANK",https://www.banki.ru/services/responses/bank/a...,49205774.0,1.653004e+09,9.240137e+10,9.270100e+10,1485435.0,https://www.sravni.ru/bank/avers/,NaN,https://otzovik.com/reviews/bank_avers_russia/,"Аверс, Аверс банк"
3,1455,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""АВТОГРАДБАНК""","АВТОГРАДБАНК, АО",Автоградбанк,"AVTOGRADBANK, AO",https://www.banki.ru/services/responses/bank/a...,49205878.0,1.650072e+09,9.243000e+10,9.273000e+10,1464906.0,https://www.sravni.ru/bank/avtogradbank/,NaN,NaN,"Автоград, Автоградбанк"
4,1973,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АВТО...","АВТОКРЕДИТБАНК, ООО",АвтоКредитБанк,"AVTOKREDITBANK, OOO",https://www.banki.ru/services/responses/bank/a...,49205795.0,1.626000e+09,9.240138e+10,9.270100e+10,1481856.0,https://www.sravni.ru/bank/avtokreditbank/,NaN,NaN,"Автокредит, Автокредитбанк"


In [4]:
irec_df = df[["id", "Наименование", "url_irec"]].dropna()
irec_df.head()

,id,Наименование,url_irec
0,2306,"АБСОЛЮТ БАНК (ПАО), АКБ",https://irecommend.ru/content/absolyut-bank
1,2879,"АВАНГАРД, ПАО АКБ",https://irecommend.ru/content/avangard
9,1810,АЗИАТСКО-ТИХООКЕАНСКИЙ БАНК (АО),https://irecommend.ru/content/aziatsko-tikhook...
12,2590,"АК БАРС БАНК, ПАО",https://irecommend.ru/content/ak-bars-bank-2
13,2587,"АКИБАНК, ПАО",https://irecommend.ru/content/pao-akibank-0


In [8]:
response = requests.get("http://localhost:8000/bank/")

In [10]:
banks_ids = [bank["id"] for bank in response.json()["items"]]

In [18]:
irec_df = irec_df[irec_df["id"].isin(banks_ids)]

In [19]:
np.save("irecommend", irec_df)

In [6]:
otzovik_df = df[["id", "Наименование", "url_otz"]].dropna()
otzovik_df.head()

,id,Наименование,url_otz
0,2306,"АБСОЛЮТ БАНК (ПАО), АКБ",https://otzovik.com/reviews/bank_absolyut_bank...
1,2879,"АВАНГАРД, ПАО АКБ",https://otzovik.com/reviews/bank_avangard_russia/
2,415,"АВЕРС, ООО БАНК",https://otzovik.com/reviews/bank_avers_russia/
9,1810,АЗИАТСКО-ТИХООКЕАНСКИЙ БАНК (АО),https://otzovik.com/reviews/aziatsko-tihookean...
11,3475,АЙСИБИСИ БАНК (АО),https://otzovik.com/reviews/bank_aysibisi_russ...


In [7]:
np.save("otzovik", otzovik_df)

# irecommend

In [ ]:
urls = []  # urls of reviews
gecko = Service(GeckoDriverManager().install())
browser = webdriver.Firefox(service=gecko)
browser.get("https://irecommend.ru")
browser.add_cookie({"name": "ss_uid", "value": "16657360693968717"})
browser.add_cookie(
    {
        "name": "stats_u_a",
        "value": "2FwvdtyzJLp9avMj9v1UDmg%2FFm0UI%2FXBW%2FsyM1AaEAw3T%2B4TNNXo5lfkEHoCwRfjzJGxf%2B8kFi7JSYfhs9N1Y7fNh6VCLOvq",
    }
)
browser.add_cookie({"name": "ab_var", "value": "7"})
browser.add_cookie(
    {"name": "stats_s_a", "value": "xFaamZkIWvN9rPje0Yyp%2FQ3JHeijRMZodVCSo0G%2FXRxy9IMakAA1QAIUJJmDZ1qw"}
)
browser.add_cookie({"name": "statsactivity", "value": "36"})
browser.add_cookie({"name": "statstimer", "value": "486"})
browser.add_cookie({"name": "v", "value": "63"})
browser.add_cookie({"name": "ss_hid", "value": "31135198"})
browser.set_page_load_timeout(5)

for id, row in tqdm(irec_df.iterrows(), total=irec_df.shape[0]):
    bank_url = row["url_irec"] + "?new=1"

    try:
        browser.get(bank_url)
    except TimeoutException:
        pass
    page = BeautifulSoup(browser.page_source, "html.parser")

    pages = 1  # pages for banks
    if page.find("ul", {"class": "pager"}) is not None:
        pages = int(page.find("ul", {"class": "pager"}).find("li", {"class": "pager-last"}).text)

    for i in tqdm(range(pages), desc=row["Наименование"], disable=True):
        if i != 0:
            try:
                browser.get(f"{bank_url}&page={i}")
            except TimeoutException:
                pass
        page = BeautifulSoup(browser.page_source, "html.parser")
        if page.find("div", id="block-quicktabs-3") is not None:
            for elem in page.find("div", id="block-quicktabs-3").findAll("div", {"class": "reviewTextSnippet"}):
                urls.append(("https://irecommend.ru" + elem.find("a", recursive=False)["href"], row["Наименование"]))
        elif page.find("h1", class_="not-found-title") is not None:
            break
    browser.close()

In [ ]:
gecko = Service(GeckoDriverManager().install())
browser = webdriver.Firefox(service=gecko)
browser.get("https://irecommend.ru")
browser.add_cookie({"name": "ss_uid", "value": "16657360693968717"})
browser.add_cookie(
    {
        "name": "stats_u_a",
        "value": "2FwvdtyzJLp9avMj9v1UDmg%2FFm0UI%2FXBW%2FsyM1AaEAw3T%2B4TNNXo5lfkEHoCwRfjzJGxf%2B8kFi7JSYfhs9N1Y7fNh6VCLOvq",
    }
)
browser.add_cookie({"name": "ab_var", "value": "7"})
browser.add_cookie(
    {"name": "stats_s_a", "value": "xFaamZkIWvN9rPje0Yyp%2FQ3JHeijRMZodVCSo0G%2FXRxy9IMakAA1QAIUJJmDZ1qw"}
)
browser.add_cookie({"name": "statsactivity", "value": "36"})
browser.add_cookie({"name": "statstimer", "value": "486"})
browser.add_cookie({"name": "v", "value": "63"})
browser.add_cookie({"name": "ss_hid", "value": "31135198"})
browser.set_page_load_timeout(5)

reviews = []

for url, name in tqdm(urls, total=len(urls)):
    try:
        browser.get(url)
    except TimeoutException:
        pass
    response = requests.get(url)
    page = BeautifulSoup(browser.page_source, "html.parser")

    res = {}
    res["url"] = url
    res["bank_name"] = name
    pg = page.find("div", {"itemprop": "review"})

    if pg is None:
        sleep(4)
        print("None")
        # browser.close()
        continue
    res["date"] = pg.find("meta", {"itemprop": "datePublished"})["content"]

    info = pg.find("li", {"class": "first last"})
    res["info"] = info.text if hasattr(info, "text") else None

    res["text"] = "".join(
        [i.text for i in pg.find("div", {"class": "description hasinlineimage", "itemprop": "reviewBody"}).findAll("p")]
    )

    positive = pg.find("div", {"class": "ratio"}).find("div", {"class": "plus"})
    res["positive"] = [i.text for i in positive.findAll("li")] if hasattr(positive, "findAll") else None

    negative = pg.find("div", {"class": "ratio"}).find("div", {"class": "minus"})
    res["negative"] = [i.text for i in negative.findAll("li")] if hasattr(negative, "findAll") else None

    res["raiting"] = len(
        pg.find("div", {"class": "fivestarWidgetStatic fivestarWidgetStatic-vote fivestarWidgetStatic-5"}).findAll(
            "div", {"class": "on"}
        )
    )
    # print(res)
    # sleep(3)
    reviews.append(res)
    # browser.close()

# Otzovik

In [8]:
gecko = Service(GeckoDriverManager().install())
base_url = "https://otzovik.com"
banks_otz = []
urls = []
browser = webdriver.Firefox(service=gecko)
browser.set_page_load_timeout(3)
browser.get("https://otzovik.com")
browser.add_cookie({"name": "ss_uid", "value": "3725698921"})
browser.add_cookie({"name": "refreg", "value": "1663846124~https%3A%2F%2Fotzovik.com%2F"})
browser.add_cookie({"name": "ROBINBOBIN", "value": "783c90df43a6a299b5b3972eff"})
browser.add_cookie({"name": "cgreg", "value": "yes"})
browser.add_cookie({"name": "ext_uid", "value": "6350ee343713b109391584"})
browser.add_cookie({"name": "gxvoid", "value": "6350ee343713b109391584"})
browser.add_cookie({"name": "yec", "value": "1"})
browser.add_cookie({"name": "ycgree", "value": "yes"})

for _, row in tqdm(otzovik_df.iterrows(), total=otzovik_df.shape[0]):
    try:
        browser.get(row["url_otz"]+"?order=date_desc")
    except TimeoutException:
        pass

    page = BeautifulSoup(browser.page_source, "html.parser")

    bank = {}
    if page.find("span", {"class": "fn"}) is None:
        break
    bank["title"] = page.find("span", {"class": "fn"}).text
    bank["rating"] = float(
        page.find("span", {"class": "product-rating tooltip-right rating"})["title"].split(":")[1].strip()
    )
    if not (page.find("span", {"class": "recommend-ratio"}) is None):
        bank["recommend_ratio"] = float(page.find("span", {"class": "recommend-ratio"}).find("span").text[:-1])

    for elem in page.find_all("div", {"class": "rating-item tooltip-top hover-brace"}):
        bank[elem.find("div", {"class": "rating-caption"}).text] = float(
            elem.find("div", {"class": "rating-fill"})["style"].split(": ")[1][:-2]
        )

    for i, elem in enumerate(page.find_all("span", {"class": "reviews-counter"})[:-1]):
        bank[i] = int(elem.find("span").text)

    banks_otz.append(bank)
    pages = 1
    if not (page.find("div", {"class": "pager"}) is None):
        if page.find("div", {"class": "pager"}).find("a", {"class": "pager-item last tooltip-top"}) is None:
            pages = page.find("div", {"class": "pager"}).find_all("a", {"class": "pager-item nth"})[-1].text
        else:
            pages = (
                page.find("div", {"class": "pager"})
                .find("a", {"class": "pager-item last tooltip-top"})["href"]
                .split("/")[-2]
            )

    for page_num in range(int(pages)):
        try:
            browser.get(f"{row['url_otz']}{page_num}/?order=date_desc")
        except TimeoutException:
            pass
        page = BeautifulSoup(browser.page_source, "html.parser")
        cards = page.find_all("div", {"class": "item status4 mshow0"})
        for card in cards:
            urls.append((base_url + card.find("a", {"class": "review-title"})["href"], row["Наименование"]))
        sleep(2)
    sleep(2)

[WDM] - ====== WebDriver manager ======
[WDM] - Current firefox version is 105.0
[WDM] - Get LATEST geckodriver version for 105.0 firefox
[WDM] - Driver [/home/samoed/.wdm/drivers/geckodriver/linux64/v0.32.0/geckodriver] found in cache


  0%|          | 0/139 [00:00<?, ?it/s]

In [57]:
gecko = Service(GeckoDriverManager().install())
reviews = []

for url, name in tqdm(urls):
    browser = webdriver.Firefox(service=gecko)
    browser.set_page_load_timeout(3)
    try:
        browser.get(url)
    except TimeoutException:
        pass

    review_data = {}
    review_data["url"] = url
    review_data["name"] = name
    user = {}
    res = {}
    ratings = {}

    page = BeautifulSoup(browser.page_source, "html.parser")

    review = page.find("div", {"class": "review-contents"})

    review_data["title"] = review.find("h1").text
    review_data["date"] = review.find("span", {"class": "review-postdate dtreviewed"}).find("abbr")["title"]
    review_data["comments"] = review.find("a", {"class": "review-btn review-comments tooltip-left hover-brace"}).text

    review_data["text"] = review.find("div", {"class": "review-body description"}).text

    for i in review.find("table", {"class": "product-props"}).find_all("tr"):
        items = i.find_all("td")
        if items[0].text != "Моя оценка:":
            res[items[0].text] = items[1].text
        else:
            res[items[0].text] = items[1].find("div", {"class": "product-rating tooltip-right"})["title"].split(":")[-1]

    for i in review.find("div", {"class": "product-rating-details"}).find_all(
        "div", {"class": "rating-item tooltip-top"}
    ):
        ratings[i["title"].split(":")[0]] = i["title"].split(":")[1][1]
    review_data |= res
    review_data |= ratings
    review_data |= user

    reviews.append(review_data)
    sleep(3)

  0%|          | 0/4810 [00:00<?, ?it/s]

NameError: name 'ua' is not defined